In [1]:
!git add .
!git commit -m "EIA Data updating script looks good to go"
!git push

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory
[master f0b4ec9] News Report is working (but not live updating)
 8 files changed, 432 insertions(+), 225 deletions(-)
 rewrite 3_testing.ipynb (67%)
 create mode 100644 scripts/bulk_download.py
To https://github.com/thekhan314/CrudeOilApp.git
   e6e8f15..f0b4ec9  master -> master


In [1]:
from data.data_functions import *
import math
import datetime as dt
import sqlite3
import pandas as pd

from datashop import *

path = 'c:/prompt_root/CrudeOilApp'
relpath = '/data/nyt_api/nyt_jsons/'

conn = sqlite3.connect('data/energydash.db')

#test_bulk = jsons_to_frame(path,relpath,conn)
#test_bulk.to_csv('data/nyt_api/nyt_articles.csv')

In [2]:
import datetime as dt

query = " SELECT MAX(Date) from MonthlyRigCount"
last_date = conn.execute(query).fetchall()[0][0]
last_date = dt.datetime.strptime(last_date[:10], '%Y-%m-%d').strftime('%Y%m%d')
print(last_date)

20200501


# EIA Data

In [3]:


# EIA_Bulk


eia_dict = {
    'PET.WTTSTUS1.W':['WeeklyStocks','235081','%Y%m%d'],
    'PET.RWTC.D':['DailyPrice','241335','%Y%m%d'],
    'PET.WRPUPUS2.W':['ProductSupplied','401676','%Y%m%d'],
    'PET.E_ERTRRO_XR0_NUS_C.M':['MonthlyRigCount','296749','%Y%m']
}

for key,val in eia_dict.items():

    e_ser = EIA_Series(key,name = val[0],date_format = val[2], end = '20200910')

    bulk_ser = e_ser.series

    bulk_ser.to_sql(val[0],conn,if_exists='replace')  

    query = " SELECT * FROM {} ORDER BY Date DESC LIMIT 5".format(val[0])
    df = pd.read_sql(query,conn)
    print(df)

                  Date  WeeklyStocks
0  2020-09-04 00:00:00     2072722.0
1  2020-09-03 00:00:00     2076448.0
2  2020-09-02 00:00:00     2076448.0
3  2020-09-01 00:00:00     2076448.0
4  2020-08-31 00:00:00     2076448.0
                  Date  DailyPrice
0  2020-09-10 00:00:00       37.25
1  2020-09-09 00:00:00       38.05
2  2020-09-08 00:00:00       36.87
3  2020-09-07 00:00:00       39.69
4  2020-09-04 00:00:00       39.69
                  Date  ProductSupplied
0  2020-09-04 00:00:00          18678.0
1  2020-09-03 00:00:00          16979.0
2  2020-09-02 00:00:00          16979.0
3  2020-09-01 00:00:00          16979.0
4  2020-08-31 00:00:00          16979.0
                  Date  MonthlyRigCount
0  2020-05-01 00:00:00            267.0
1  2020-04-30 00:00:00            471.0
2  2020-04-29 00:00:00            471.0
3  2020-04-28 00:00:00            471.0
4  2020-04-27 00:00:00            471.0


In [12]:
eia_dict = {
    'PET.WTTSTUS1.W':['WeeklyStocks','235081','%Y%m%d'],
    'PET.RWTC.D':['DailyPrice','241335','%Y%m%d'],
    'PET.WRPUPUS2.W':['ProductSupplied','401676','%Y%m%d'],
    'PET.E_ERTRRO_XR0_NUS_C.M':['MonthlyRigCount','296749','%Y%m']
}

with open ('data/cache.json','r') as cache_file:
    cache_dict = json.load(cache_file)

#_________ Base URL. Requires category ID to return series in that category"

eia_api_url= 'http://api.eia.gov/updates/?api_key=651b30b69f4f47a13a2912d673f7da93&category_id='



# ________ Get last update time for each series. 

current_eia = {}

for key,val in eia_dict.items():

    url = eia_api_url + val[1] + '&rows=10000'
    request = requests.get(url)
    dict = json.loads(request.text)

    # Load last updated time from DB
    
    query = " SELECT MAX(Date) from {}".format(val[0])
    last_date = conn.execute(query).fetchall()[0][0]
    last_date_eia = dt.datetime.strptime(last_date[:10], '%Y-%m-%d').strftime(val[2])

    last_update = 'not found'
    #Get last updated time from API
    for update in dict['updates']:
        if update['series_id'] == key:
            last_update = update['updated']
            last_update = dt.datetime.strptime(last_update[:10], '%Y-%m-%d').strftime(val[2])

    #compare both, get updates if needed and add to table

    if last_update > last_date:
        new_series = EIA_Series(
            key, 
            name=val[0],
            start = last_date_eia,
            date_format = val[2]).series

        new_series = new_series[new_series.index > pd.Timestamp(last_date)]
        new_series.to_sql(val[0],conn,if_exists='append')   

    # get the current last value for later use

    query = " SELECT * FROM  {} WHERE Date = (SELECT MAX(Date) FROM {})".format(val[0],val[0])
    last_date = conn.execute(query).fetchall()[0]

    current_eia[val[0]] = last_date

print(current_eia)


{'WeeklyStocks': ('2020-10-30 00:00:00', 2015817.0), 'DailyPrice': ('2020-11-03 00:00:00', 37.44), 'ProductSupplied': ('2020-10-30 00:00:00', 18362.0), 'MonthlyRigCount': ('2020-05-01 00:00:00', 267.0)}


In [13]:
print(pd.Timestamp(last_date))

2020-10-30 00:00:00


In [ ]:
if last_update > last_date:
        new_series = EIA_Series(key,start=last_date)
        new_series.series.to_sql(val[0],conn,if_exists='append')

# News Data 

In [ ]:
c = conn.cursor()
last_date = c.execute('SELECT max(retrieved) from news').fetchone()[0]
print(last_date)

In [ ]:
with open ('data/cache.json','r') as cache_file:
    cache_dict = json.load(cache_file)

start_date = cache_dict['news_update']
end_date = str(dt.datetime.now())

quer = 'Oil (Petroleum) and Gasoline'

test_call = nytResp(start_date,end_date,quer)

if test_call.hits > 0:
    conn = sqlite3.connect('data/energydash.db')
    test_call.frame.to_sql('news',conn,if_exists='append')

with open ('data/cache.json','w') as cache_file:
    json.dump(test_call.cache_dict,cache_file)

In [ ]:
# Testing News

conn = sqlite3.connect('data/energydash.db')

query = 'SELECT date,abstract,url,retrieved FROM news ORDER BY date DESC LIMIT 10'
df = pd.read_sql_query(query,conn)